In [ ]:
import gym
import numpy as np
import random

In [ ]:
env= gym.make("MountainCar-v0")

In [ ]:
low = env.observation_space.low
high = env.observation_space.high


In [ ]:
def create_grids(l,h,bins=(10,10)):
    grid = [np.linspace(l[dim],h[dim],bins[dim]+1)[1:-1] for dim in range(len(bins))]
    return grid


In [ ]:
def discrete(sample_state, grid):
    return list(int(np.digitize(s,g)) for s, g in zip(sample_state, grid))

In [ ]:
state_grid = create_grids(env.observation_space.low, env.observation_space.high, bins=(10, 10))

In [ ]:
class my_agent():
    
    def __init__(self,grid,env):
            self.env = env
            self.grid = grid
            self.state_size = tuple(len(i)+1 for i in self.grid)
            print("State space size:", self.state_size)
            self.action_size = env.action_space.n
            self.gamma = 0.99
            self.alpha = 0.02
            self.epsilon = 1.0
            self.decay = 0.0001 
            self.min_epsilon = 0.01
            self.q_table = np.zeros(shape=(self.state_size + (self.action_size,)))
            print("Q table size:", self.q_table.shape)
        
    def get_state(self,state):
            return tuple(discrete(state, self.grid))
                     
                                          
    def _update_epsilon(self):
            self.epsilon -= self.decay
            self.epsilon = max(self.min_epsilon, self.epsilon)

    def act(self, state, is_new_episode):
        
#             if is_new_episode:
#                 return self.env.action_space.sample()
                
                
        
            if np.random.rand() <= self.epsilon:
                         self._update_epsilon()
                         return self.env.action_space.sample()
                         
            state =  self.get_state(state)
            action = np.argmax(self.q_table[state])
            return action             
                         
    def q_update(self,state, action, reward, next_state):
                         state =  self.get_state(state)
                         next_state = self.get_state(next_state)
                         self.q_table[state + (action,)] += self.alpha * \
                        (reward + self.gamma * max(self.q_table[next_state]) - \
                         self.q_table[state + (action,)])
                         
agent = my_agent(state_grid, env)                         

In [ ]:
total_episode = 20_000
i = 0
for ep in range(total_episode):
                done = False
                state = env.reset()
                total_reward = 0
                action = agent.act(state,True)

                
                i +=1
                while not done:
                    
                    env.render()
                    next_state, reward, done, _ = env.step(action)
                    agent.q_update(state, action, reward, next_state)
                    total_reward += reward   
                    state = next_state    
                    action = agent.act(state, False)
                if done:
                           print("total reward of episode {} is {}".format(i,total_reward))
                           

